### 국내 주식 주별 시세 UPDATE

In [4]:
import requests
import json
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
import os

In [31]:
url = "/uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice"

load_dotenv()

my_app= os.getenv("MY_APP")
my_sec= os.getenv("MY_SEC")

#계좌번호 앞 8자리
my_acct="43016876"
my_acct_stock= "증권계좌 8자리"
my_acct_future= "선물옵션계좌 8자리"
#계좌번호 뒤 2자리
my_prod= "01"
#my_prod: "03"

#실전투자
prod= "https://openapi.koreainvestment.com:9443"
#모의투자
vps= "https://openapivts.koreainvestment.com:29443"


my_agent= "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"

class Env:
    def __init__(self):
        self.my_app = my_app
        self.my_sec = my_sec
        self.my_url = prod
        self.my_token = None
        self.my_acct = my_acct
        self.my_prod = my_prod
_env = Env()

In [57]:
def auth():
    url = f'{_env.my_url}/oauth2/tokenP'
    headers = {"content": "application/json"}
    data = {
        "grant_type":"client_credentials",
        "appkey": _env.my_app,
        "appsecret": _env.my_sec
    }

    res = requests.post(url, headers=headers, data= json.dumps(data))
    if res.status_code == 200:
        print('토큰 발급 성공')
        _env.my_token = res.json()['access_token']
    else:
        print("토큰 발급 실패")
        print(res.status_code, res.text)

In [58]:
auth()

토큰 발급 성공


KOSPI200 종목에 대해 주별 시세 받아오기

In [125]:
stock_code = "005930"
market_code = "J"

In [126]:
headers = {
    "Content-Type": "application/json",
    "authorization": f'Bearer {_env.my_token}',
    "appkey": _env.my_app,
    "appsecret": _env.my_sec,
    "tr_id": "FHKST03010100", 
    "custtype": "P"
}

In [127]:
params = {
    "fid_cond_mrkt_div_code": market_code,  
    "fid_input_iscd": stock_code,
    "fid_period_div_code": "W"
}

In [128]:
START = 2015
END = 2024

In [129]:
params['fid_input_date_1'] = '19660101'
params['fid_input_date_2'] = '19661231'
params['fid_org_adj_prc'] = 0

In [130]:
print(params)

{'fid_cond_mrkt_div_code': 'J', 'fid_input_iscd': '005930', 'fid_period_div_code': 'W', 'fid_input_date_1': '19660101', 'fid_input_date_2': '19661231', 'fid_org_adj_prc': 0}


In [131]:
my_url=_env.my_url+url
response = requests.get(my_url, headers=headers, params=params)
if response.status_code != 200:
    print('조회 실패')
else:
    print('조회 성공')
    data = response.json()

조회 성공


{'output1': {'prdy_vrss': '400',
  'prdy_vrss_sign': '2',
  'prdy_ctrt': '0.73',
  'stck_prdy_clpr': '54700',
  'acml_vol': '10867069',
  'acml_tr_pbmn': '597524532250',
  'hts_kor_isnm': '삼성전자',
  'stck_prpr': '55100',
  'stck_shrn_iscd': '005930',
  'prdy_vol': '14437025',
  'stck_mxpr': '71100',
  'stck_llam': '38300',
  'stck_oprc': '54700',
  'stck_hgpr': '55300',
  'stck_lwpr': '54500',
  'stck_prdy_oprc': '56000',
  'stck_prdy_hgpr': '56200',
  'stck_prdy_lwpr': '54500',
  'askp': '55200',
  'bidp': '55100',
  'prdy_vrss_vol': '-3569956',
  'vol_tnrt': '0.18',
  'stck_fcam': '100',
  'lstn_stcn': '5919637922',
  'cpfn': '7780',
  'hts_avls': '3261720',
  'per': '11.13',
  'eps': '4950.00',
  'pbr': '0.95',
  'itewhol_loan_rmnd_ratem name': '0.22'},
 'output2': [],
 'rt_cd': '0',
 'msg_cd': 'MCA00000',
 'msg1': '정상처리 되었습니다.'}

In [133]:
li = response.json()['output2']
df = pd.DataFrame(li)
df.sort_values(by='stck_bsop_date')
df = df[['stck_bsop_date', 'stck_clpr', 'stck_oprc', 'stck_hgpr', 'stck_lwpr', 'acml_vol', 'acml_tr_pbmn']]

df = df.rename(columns={
    "stck_bsop_date": "date",
    "stck_clpr": "close",
    "stck_oprc": "open",
    "stck_hgpr": "high",
    "stck_lwpr": "low",
    "acml_vol": "volume",
    "acml_tr_pbmn": "trade_amount"
})
df

KeyError: 'stck_bsop_date'

In [115]:
raw = response.json()['output1']

parsed = {
    "code": raw["stck_shrn_iscd"],
    "face_value": int(raw["stck_fcam"]),
    "listed_shares": int(raw["lstn_stcn"]),
    "capital": int(raw["cpfn"]),
    "market_cap": int(raw["hts_avls"]),
    "per": float(raw["per"]),
    "eps": float(raw["eps"]),
    "pbr": float(raw["pbr"]),
    "loan_balance_ratio": float(raw["itewhol_loan_rmnd_ratem name"])
}

parsed

{'code': '005930',
 'face_value': 100,
 'listed_shares': 5919637922,
 'capital': 7780,
 'market_cap': 3261720,
 'per': 11.13,
 'eps': 4950.0,
 'pbr': 0.95,
 'loan_balance_ratio': 0.22}